In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from time import sleep
import csv
import re
import json

In [2]:
# url to get artists
base_url_artists = "https://www.vagalume.com.br/browse/{}.html"

In [13]:
# To store the artist and it's link
artist_links = []
artist = []

    
artist_url = base_url_artists.format('a')
print("Going to url : ", artist_url)

# Reading the page data 
req = Request(artist_url, headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
html_page = urlopen(req).read()
soup = BeautifulSoup(html_page, 'html.parser')
    
# Finding the required attributes
artist_list = soup.find('ul', class_="xsList2 smList4 mdList6 gridList", id="artTopsSection").find_all('a', href=True)
# Formatting and storing the information
for a in artist_list:
    artist_links.append('https://www.vagalume.com.br' + a['href'])
    artist.append(a.text)
        
# Testing 
# print(list(zip(artist_links,artist)))

Going to url :  https://www.vagalume.com.br/browse/a.html


In [14]:
print(list(zip(artist_links,artist)))

[('https://www.vagalume.com.br/adele/', 'Adele'), ('https://www.vagalume.com.br/anitta/', 'Anitta'), ('https://www.vagalume.com.br/aline-barros/', 'Aline Barros'), ('https://www.vagalume.com.br/alicia-keys/', 'Alicia Keys'), ('https://www.vagalume.com.br/ariana-grande/', 'Ariana Grande'), ('https://www.vagalume.com.br/aerosmith/', 'Aerosmith'), ('https://www.vagalume.com.br/andre-valadao/', 'André Valadão'), ('https://www.vagalume.com.br/arctic-monkeys/', 'Arctic Monkeys'), ('https://www.vagalume.com.br/ac-dc/', 'AC/DC'), ('https://www.vagalume.com.br/a-ha/', 'a-ha'), ('https://www.vagalume.com.br/abba/', 'ABBA'), ('https://www.vagalume.com.br/amy-winehouse/', 'Amy Winehouse'), ('https://www.vagalume.com.br/alok/', 'Alok'), ('https://www.vagalume.com.br/avril-lavigne/', 'Avril Lavigne'), ('https://www.vagalume.com.br/avicii/', 'AVICII'), ('https://www.vagalume.com.br/ademar-de-campos/', 'Adhemar de Campos'), ('https://www.vagalume.com.br/andrea-bocelli/', 'Andrea Bocelli'), ('https://w

In [15]:
# Storing the song dictionary
songs_dict = {} 

for url, artst in zip(artist_links, artist):

    print("Going to url : ", url)

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
    html_page = urlopen(req).read()
    soup = BeautifulSoup(html_page, 'html.parser')
    
    genre_list = soup.find('div', class_="scrollable").find_all('a', href=True)
    genres_list = []
    for genre in genre_list:
        genres_list.append(genre.text)
#     print(genres_list)
    
    songs_dict[artst] = {'genres':genres_list, 'albums': []}
    
    popularity_req = Request(url + "popularidade/", headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
    popularity_html_page = urlopen(popularity_req).read()
    popularity_soup = BeautifulSoup(popularity_html_page, 'html.parser')
    
    popularity = popularity_soup.find('ul', id='popFacts')
#     print(popularity)
    
    albums_req = Request(url + "discografia/", headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
    albums_html_page = urlopen(albums_req).read()
    albums_soup = BeautifulSoup(albums_html_page, 'html.parser')
    
    albums_list = albums_soup.find('div', class_='letrasWrapper').find_all('div', class_='cardAlbumInfos')
    
    albums_data = []
    for album in albums_list:
        album_data = {
                        'albumTitle': album.find('h1', {'class': 'albumTitle'}).find('a').text,
                        'albumLink': album.find('a')['href'],
                        'albumYear': '',
                        'albumRecord': '',
                        'songs': []
                    }
        album_year_record = album.find('p', {'class': 'albumYear'}).get_text(separator='---')
        if '---' in album_year_record:
            album_data['albumYear'], album_data['albumRecord'] = album_year_record.split('---')
        else:
            album_data['albumYear'] = album_year_record.strip()
            album_data['albumRecord'] = ''
        
        albums_songs_req = Request('https://www.vagalume.com.br' + album_data['albumLink'], headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
        print('https://www.vagalume.com.br' + album_data['albumLink'])
        albums_songs_req_html_page = urlopen(albums_songs_req).read()
        albums_songs_req_soup = BeautifulSoup(albums_songs_req_html_page, 'html.parser')
        
        songs_list = albums_songs_req_soup.find('ol', id='topMusicList')
    
        if songs_list:
            songs_list = songs_list.find_all('a', class_='nameMusic', href=True)
            count = len(songs_list)
#             print(songs_list, count)

            for song in songs_list:
                song_dict = {}
                song_dict['name'] = song.text
                song_dict['lyrics_link'] = 'https://www.vagalume.com.br' + song['href']
                req = Request(song_dict['lyrics_link'], headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
                html_page = urlopen(req).read()
                soup = BeautifulSoup(html_page, 'html.parser')
                song_dict['lyrics'] = soup.find('div', id='lyrics').get_text(separator='. ').replace('<br/>', '. ')
#                 print(song_dict)
                album_data['songs'].append(song_dict)
                
        else:
            print(f"No songs found for {artst}. Removing the url {url} from the list")
            artist_links.remove(url)
            artist.remove(artst)

        sleep(1)
#         print(album_data)
        albums_data.append(album_data)
    
# print(albums_data)

    songs_dict[artst]['albums'].append(albums_data)
    break
    
    
    
    
    
    
    
    
#     songs_list = soup.find('ol', id='alfabetMusicList')
    
#     if songs_list:
#         songs_list = songs_list.find_all('a', class_='nameMusic', href=True)
#         count = len(songs_list)
#         songs_dict[artst] = {'genres':genres_list, 'count': count, 'songs': []}
#         for song in songs_list:
#             song_dict = {}
#             song_dict['name'] = song.text
#             song_dict['lyrics_link'] = 'https://www.vagalume.com.br' + song['href']
#             req = Request(song_dict['lyrics_link'], headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'})
#             html_page = urlopen(req).read()
#             soup = BeautifulSoup(html_page, 'html.parser')
#             song_dict['lyrics'] = soup.find('div', id='lyrics').get_text(separator='. ').replace('<br/>', '. ')
#             songs_dict[artst]['songs'].append(song_dict)
#     else:
#         print(f"No songs found for {artst}. Removing the url {url} from the list")
#         artist_links.remove(url)
#         artist.remove(artst)

#     sleep(1)
#     break
# print(songs_dict)

Going to url :  https://www.vagalume.com.br/adele/
https://www.vagalume.com.br/adele/discografia/30.html
https://www.vagalume.com.br/adele/discografia/25-11.html
https://www.vagalume.com.br/adele/discografia/live-at-the-royal-albert-hall-11.html
https://www.vagalume.com.br/adele/discografia/21-11.html
https://www.vagalume.com.br/adele/discografia/19.html


In [16]:
songs_dict

{'Adele': {'genres': ['Soul Music', 'Pop', 'R&B'],
  'albums': [[{'albumTitle': '30',
     'albumLink': '/adele/discografia/30.html',
     'albumYear': '2021',
     'albumRecord': 'Sony Music Entertainment',
     'songs': [{'name': 'Strangers By Nature',
       'lyrics_link': 'https://www.vagalume.com.br/adele/strangers-by-nature.html',
       'lyrics': "I'll be taking flowers to the cemetery of my heart. For all of my lovers, in the present and in the dark. Every anniversary I'll pay respects and say I'm sorry. For they never stood a chance as if they could. When no one knows what it's like to be us. I've never seen the sky this colour before. It's like I'm noticing everything a little bit more. Now that all the dust has settled. I rebut all my rebuttals. No one knows what it's like to be us. Strangers by nature. Strangers by nature. Will I ever get there?. Oh, I hope that someday I'll learn. To nurture what I've done. Uh, huh, hm. Uh, huh, hm, hm, hm. Uh, huh, hm, hm, hm. Hm, hm, hm,

In [17]:
# open a file in write mode
with open("adele_songs.json", "w") as f:
    # write the dictionary to the file in JSON format
    json.dump(songs_dict, f)